In [ ]:
import os
import re

import numpy as np
import pandas as pd
from scipy.optimize import minimize_scalar

import gvar as gv

import seaborn as sns
import matplotlib.pylab as plt

from luescher_nd.database import utilities as ut
from luescher_nd.database.utilities import DATA_FOLDER
from luescher_nd.zeta.extern.pyzeta import zeta
from luescher_nd.zeta.extern.pyzeta import zeta as spherical_zeta

from luescher_nd.plotting import styles

from luescher_nd.database.utilities import get_continuum_extrapolation

from luescher_nd.hamiltonians.kinetic import MomentumKineticHamiltonian
from luescher_nd.solvers.contact import FitKernel

from luescher_nd.database.utilities import get_degeneracy

from luescher_nd.plotting.styles import MARKERS
from luescher_nd.database.utilities import get_degeneracy

from luescher_nd.database.utilities import _poly as polyfcn

import matplotlib

styles.setup(pgf=False)

In [ ]:
A1_WIDTH = 6.0

matplotlib.use("pgf")
sns.set(
    context="paper",
    style="ticks",
    font_scale=1/1.7,
    rc={
        # "mathtext.fontset": "cm",
        "pgf.rcfonts": False,
        "axes.unicode_minus": False,
        "font.serif": [],
        "font.sans-serif": [],
    },
)

In [ ]:
%load_ext blackcellmagic

In [ ]:
a_inv = 0.0
L = 1.0

In [ ]:
dummy = MomentumKineticHamiltonian(1)
kernel = FitKernel(dummy, spherical_zeta, 0.0)

In [ ]:
poles = list(get_degeneracy(10).keys())

zeros = {}
for n, pole in enumerate([-20] + poles):
    if n == len(poles):
        break
    start, end = pole, poles[n]
    zero = kernel.get_zeta_intersection(
        bounds=(start + 1.0e-2, end - 1.0e-2),
        method="bounded",
        options={"xatol": 1.0e-16},
    )
    zeros[n] = zero

In [ ]:
files = [f for f in os.listdir(DATA_FOLDER) if f.endswith(".sqlite") and not "tmp" in f]
files

In [ ]:
file_name = f"contact-fitted_a-inv={a_inv:+1.1f}_zeta=spherical_projector=a1g_n-eigs=200.sqlite"
print(file_name)
df = ut.read_table(
    os.path.join(DATA_FOLDER, file_name),
    zeta=None,
    round_digits=2,
    filter_poles=False,
    filter_by_nstates=False,
    filter_degeneracy=False,
).query("nlevel < 24 and L == @L and n1d > 5")[
    ["n1d", "epsilon", "nstep", "L", "x", "nlevel", "mass"]
]
df["L"] = df.L.round(7)
df.head()

def nstep_label(nstep) -> str:
    return "$" + (str(nstep) if nstep > 0 else r"\infty") + "$"

df["nstep_label"] = df.nstep.apply(nstep_label)

In [ ]:
df.epsilon.unique().size

In [ ]:
fit = get_continuum_extrapolation(df, n_poly_max=6, odd_poly=True).query("n_poly_max > 1")

In [ ]:
def avg_gvars(ys):
    y_mean = y_sdev = 0
    len_ys = len(ys) 
        
    for y in ys:
        y_mean += y / len_ys
    for y in ys:
        y_sdev += (y - y_mean)**2 / len_ys
        

    y_mean += gv.gvar([0]*len(y_sdev), np.sqrt(gv.mean(y_sdev)))

    return y_mean

In [ ]:
def plot_frame(*args, **kwargs):
    x_key, y_key = args
    df = kwargs.pop("data")
    eps = df[x_key].values
    x = df[y_key].values
    
    nstep = df.nstep.unique()[0]
    nlevel = df.nlevel.unique()[0]
    L = df.L.unique()[0]
    fits = fit.query("L == @L and nlevel == @nlevel and nstep == @nstep")
    
        
    ax = plt.gca()
    ax.plot(eps, x, **kwargs)
    
    ys = []
    xs = []
    eps = np.linspace(1.e-3, eps.max())
    for idx, fit_data in fits.iterrows():
        x = []
        xs.append(np.array([fit_data["x0"]]))
        for key, val in fit_data.items():
            if re.match("x[0-9]+", key):
                if not pd.isna(val):
                    x.append(val)
        p = {"x": x}
        ys.append(polyfcn(eps, p, even=False))
    
    y = avg_gvars(ys)
    x = avg_gvars(xs)
    
    
    ax.plot(eps, gv.mean(y), color="green")
    ax.fill_between(eps, gv.mean(y)-gv.sdev(y), gv.mean(y)+gv.sdev(y), color="green", alpha=.5,zorder=-3)

    ax.axhline(zeros[nlevel], color="black", lw=1, ls="--",zorder=-2)
    ax.axhline(gv.mean(x), color="blue", lw=0.5,zorder=-3, alpha=0.8)
    ax.axhspan(
        gv.mean(x)-gv.sdev(x),
        gv.mean(x)+gv.sdev(x),
        color="blue",
        alpha=0.1,
        zorder=-4
    )

In [ ]:
eps = np.linspace(1.0e-3, df.epsilon.max(), 100)

grid = sns.FacetGrid(
    data=df.query("nlevel < 8 and nlevel > 0").sort_values("x", ascending=True),
    col="nstep",
    row="nlevel",
    sharex=True,
    sharey="row",
    legend_out=True,
    hue_kws={
        "marker": MARKERS * 5,
        "ms": [2] * 40,
        "lw": [0.5] * 40,
        "ls": ["None"]*40,
        "color": ["black"]*40,
        "zorder": [3]*40,
    },
    margin_titles=True,
    col_order=[nstep for nstep in [1, 2, 4, -1]],
    row_order=[nlevel for nlevel in range(1,8)[::-1]],
    xlim=(eps.min(), eps.max()*1.1),
    aspect=2, height=1.5
)

grid.map_dataframe(plot_frame, "epsilon", "x")

for ax in grid.axes.flat:
    ax.set_xscale("log")
    
grid.set_ylabels(r"$x = \frac{2 \mu E L^2}{4 \pi^2}$")
grid.set_xlabels(r"$\epsilon [\mathrm{fm}]$")
grid.set_titles(
    #row_template=r"${row_var} = {row_name} [\mathrm{{fm}}]$",
    col_template=r"$n_{{\mathrm{{step}}}} =$ {col_name}",
)

plt.subplots_adjust(wspace=0.2, hspace=0.2)

ratio = grid.fig.get_figheight() / grid.fig.get_figwidth()
grid.fig.set_figheight(ratio * A1_WIDTH)
grid.fig.set_figwidth(A1_WIDTH)


grid.fig.set_dpi(300)

In [ ]:
x_mean = np.sum(tf.x0 * tf.P)
x_sdev = np.sqrt(np.sum(tf.P*(tf.x0 - x_mean)**2)/len(tf.x0))
x_mean, x_sdev

In [ ]:
plt.axhline(gv.mean(x_mean))
plt.axhspan(gv.mean(x_mean) - gv.sdev(x_mean), gv.mean(x_mean) + gv.sdev(x_mean),alpha=0.6)
plt.axhspan(
    gv.mean(x_mean) - gv.mean(gv.sdev(x_mean) + x_sdev),
    gv.mean(x_mean) + gv.mean(gv.sdev(x_mean) + x_sdev),
    alpha=0.5,
)
plt.errorbar(
    tf.n_poly_max,
    gv.mean(tf.x0.values),
    gv.sdev(tf.x0.values),
    ls="None",
    marker=".",
    capsize=4,
)
plt.axhline(zeros[nlevel], color="black", lw=1, ls="--")


In [ ]:
tf[["x0", "P", "n_poly_max"]]